In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)

In [2]:
df = pd.read_csv('/Users/matthew/Documents/citadel/Consumer_Airfare_Report__Table_6_-_Contiguous_State_City-Pair_Markets_That_Average_At_Least_10_Passengers_Per_Day.csv')

/var/folders/bw/5y6vvgld6yv27r1hv8xx5dq80000gn/T/ipykernel_48515/3306953606.py:1: DtypeWarning: Columns (17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/matthew/Documents/citadel/Consumer_Airfare_Report__Table_6_-_Contiguous_State_City-Pair_Markets_That_Average_At_Least_10_Passengers_Per_Day.csv')


In [3]:
small_markets = df[df['passengers'] < 100]
small_markets = small_markets[['Year', 'nsmiles', 'fare', 'carrier_lg', 'passengers']]

In [4]:
# Calculate how many passengers each carrier serviced
airline_count = {a:0 for a in ['AA', 'DL', 'WN', 'UA', 'AS', 'B6', 'NK']}

for i, row in small_markets.iterrows():
    if row['carrier_lg'] in airline_count:
        airline_count[row['carrier_lg']] += row['passengers']

airline_count

{'AA': 2558092,
 'DL': 4302975,
 'WN': 1084044,
 'UA': 1670437,
 'AS': 279819,
 'B6': 28734,
 'NK': 28025}

In [6]:
small_markets = df[df['passengers'] < 100]
small_markets = small_markets[['Year', 'nsmiles', 'fare', 'carrier_lg']]

In [7]:
shift = 2004
unique_airline = small_markets['carrier_lg'].unique()
d = [{a:[] for a in unique_airline} for i in range(2004, 2023)]

In [8]:
# extract data about flights into dictionary format
for year in range(2004, 2023):
    tmp_df = small_markets[small_markets['Year'] == year]

    for airline in unique_airline:
        unqiue_df = tmp_df[tmp_df['carrier_lg'] == airline]

        for _, row in unqiue_df.iterrows():
            d[year-shift][airline].append((row['nsmiles'], row['fare']))

In [9]:
pricing = [{a:[] for a in unique_airline} for i in range(2004, 2023)]

In [10]:
# Calculate regressions for each instance of dataset
for year in range(2004, 2023):
    for airline in unique_airline:
        if len(d[year-shift][airline]) == 0: continue
        dist, fare = zip(*d[year-shift][airline])

        a, b = np.polyfit(dist, fare, 1)
        pricing[year-shift][airline].extend([a,b])


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3508: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3508: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3508: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3508: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3508: RankWarni

In [11]:
# clean data and remove empty values
major_airline_pricing = {year:{} for year in range(2004, 2023)}

for year in range(2004, 2023):
    for airline in ['AA', 'UA', 'B6', 'AS', 'NK', 'WN', 'DL']:
        try:
            major_airline_pricing[year][airline] = pricing[year-shift][airline]
        except:
            major_airline_pricing[year][airline] = []

In [12]:
small_mkt = pd.DataFrame(major_airline_pricing)
small_mkt.to_csv('small_mkt.csv')